In [14]:
%matplotlib inline

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as st
from ggplot import *
import datetime

In [16]:
turnstile_weather = pd.read_csv('./improved-dataset/turnstile_weather_v2.csv')

# 1. Non-Directional (two-tail) test

In [97]:
rain = turnstile_weather[turnstile_weather.rain == 1]
no_rain = turnstile_weather[turnstile_weather.rain == 0]
u_value, p_value = st.mannwhitneyu(rain.ENTRIESn_hourly, no_rain.ENTRIESn_hourly)
p_value *= 2
print 'u-value:{}    p-value:{}'.format(u_value, p_value)
print 'Rainy mean: {}     Non-Rainy mean:{}'.format(rain.ENTRIESn_hourly.mean(), no_rain.ENTRIESn_hourly.mean())

u-value:153635120.5    p-value:5.48213914249e-06
Rainy mean: 2028.19603547     Non-Rainy mean:1845.53943866


In [99]:
alpha = 0.05 # Significant value
print ('Reject the null' if p_value < alpha else 'Fail to reject the null')
print 'Mean difference: {}'.format(rain.ENTRIESn_hourly.mean() - no_rain.ENTRIESn_hourly.mean())

Reject the null
Mean difference: 182.656596808


# 2. Non-Directional (two-tail) test for each station

In [100]:
stations = turnstile_weather.groupby('station')

test_stations = pd.DataFrame(columns=['rain-count', 'no-rain-count', 'rain-mean', 'no-rain-mean', 'mean_diff', 't-statistic', 'p-value', 'Reject the null?'])

for name, frame in stations:
    rain = frame[frame.rain == 1]
    no_rain = frame[frame.rain == 0]
    try:
        t_statistic, p_value = st.mannwhitneyu(rain.ENTRIESn_hourly, no_rain.ENTRIESn_hourly)
        p_value *= 2
        test_stations.loc[name] = [rain.ENTRIESn_hourly.count(), no_rain.ENTRIESn_hourly.count(),rain.ENTRIESn_hourly.mean(), no_rain.ENTRIESn_hourly.mean(), rain.ENTRIESn_hourly.mean() - no_rain.ENTRIESn_hourly.mean(), t_statistic, p_value, p_value < 0.05]
    except:
        continue

In [101]:
reject_null_n = len(test_stations[(test_stations['Reject the null?'] == True) & (test_stations['rain-mean'] > test_stations['no-rain-mean'])])
print 'Nº stations that "Reject the null": {}'.format(reject_null_n)
print 'Total number of stations: {}'.format(len(test_stations))

Nº stations that "Reject the null": 1
Total number of stations: 206


In [102]:
test_stations[test_stations['Reject the null?'] == True]

,rain-count,no-rain-count,rain-mean,no-rain-mean,mean_diff,t-statistic,p-value,Reject the null?
191 ST,35,137,1244.942857,963.321168,281.621689,1882,0.049901,True
ORCHARD BEACH,32,56,0.000000,134.982143,-134.982143,656,0.001518,True


In [103]:
test_stations

,rain-count,no-rain-count,rain-mean,no-rain-mean,mean_diff,t-statistic,p-value,Reject the null?
1 AVE,42,143,3474.619048,3087.804196,386.814852,2676.5,0.285311,False
103 ST-CORONA,41,141,2668.268293,2577.503546,90.764747,2751.0,0.639681,False
104 ST,40,130,394.750000,378.269231,16.480769,2427.5,0.527378,False
110 ST,41,138,1910.146341,1838.463768,71.682573,2763.0,0.822107,False
110 ST-CPN,40,132,1439.125000,1399.189394,39.935606,2617.0,0.935003,False
111 ST,78,261,902.961538,923.080460,-20.118921,9968.0,0.781646,False
121 ST,42,133,329.404762,334.458647,-5.053885,2741.0,0.857173,False
125 ST,81,275,2652.234568,2510.505455,141.729113,11059.5,0.924154,False
135 ST,42,143,2210.976190,2022.979021,187.997169,2845.0,0.605710,False
138 ST-GR CONC,38,128,483.973684,485.101562,-1.127878,2410.5,0.935667,False
